In [2]:
import pandas as pd
import numpy as np
from music21 import converter, instrument, chord, note, stream
import os

In [3]:
save_dir = './mixed_midi/'

songList = os.listdir(save_dir)

original_scores = []

for song in songList:
    score = converter.parse(save_dir+song)
    original_scores.append(score)
    print(str(song))

AC POP CHRD 1 _ Abmaj, Abmaj7, Gmin7, Gbdom7, Fmin7, Bbdom13, Ebmaj, Ebdom7, Abmaj, Abmaj7, Gmin7, Gbdom7, Fmin7, Bbdomb9, Ebsus2.mid
AC POP CHRD 1 _ Amaj, Bmin, Csus2, Dmaj, Amaj, Bmin, Csus2, Dmaj, Edom7.mid
AC POP CHRD 1 _ Cmin, Fmin7, Bbdom7, Ebmaj, G7_D.mid
AC POP CHRD 10 _ G#min7, Emaj, Bmaj, F#maj, G#min, Emaj, Bmaj, F#maj.mid
AC POP CHRD 11 _ Emin7, Amaj, Dmaj, Gmaj.mid
AC POP CHRD 12 _ Gmin, Bbmaj, Ebmaj, Bbmaj, Ebmaj, Gmin, Fmaj, Bbmaj.mid
AC POP CHRD 13 _ G#min, Bmaj, Emaj7, F#maj, G#min, Bmaj, Emaj7.mid
AC POP CHRD 2 _ Dmaj, D7_C, Gmaj_B, Gmin7, Dmaj_A, Bmin7, E7, Adom11, Dmaj, D7_C, Gmaj_B, Gmin7, Dmaj_A, B7, E7, Adom11b9.mid
AC POP CHRD 2 _ Fmaj, Amin7, Bbmaj, Fmaj, Amin7, Bbmaj.mid
AC POP CHRD 2 _ Gmaj, Dmaj_F#, Emin, Bmin_D, Cmaj, Gmaj_B, Amin7, Ddom9.mid
AC POP CHRD 3 _ Bbmaj, Cmin7.mid
AC POP CHRD 3 _ C#min, G#min, Amaj, Amaj, Bmaj, C#min7, C#min, G#min, Amaj7, F#min, G#min, Amaj7.mid
AC POP CHRD 3 _ Emin, Emin7_D, Cmaj7, Ddom13.mid
AC POP CHRD 4 _ Amaj9, Bmaj6, C#min

In [4]:
original_scores[0].show('midi')

In [6]:
original_scores = [song.chordify() for song in original_scores]

In [7]:
original_chords = [[] for _ in original_scores]
original_durations = [[] for _ in original_scores]
original_keys = []

In [8]:
for i, song in enumerate(original_scores):
    original_keys.append((str(song.analyze('key'))))
    
    for element in song:
        if isinstance(element, note.Note):
            original_chords[i].append(element.pitch)
            original_durations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            original_chords[i].append('.'.join(str(n) for n in element.pitches))
            original_durations[i].append(element.duration.quarterLength)

In [9]:
unique_chords = np.unique([i for s in original_chords for i in s])
chord_to_int = dict(zip(unique_chords, list(range(0, len(unique_chords)))))

In [10]:
unique_durations = np.unique([i for s in original_durations for i in s])
duration_to_int = dict(zip(unique_durations, list(range(0, len(unique_durations)))))

In [11]:
int_to_chord = {i: c for c, i in chord_to_int.items()}
int_to_duration = {i: c for c, i in duration_to_int.items()}

In [12]:
sequenceLength = 32

# Define empty array for train data
train_chords = []
train_durations = []
target_chords = []
target_durations = []

# Construct train and target sequences for chords and durations
for s in range(len(original_chords)):
    chordList = [chord_to_int[c] for c in original_chords[s]]
    durationList = [duration_to_int[d] for d in original_durations[s]]
    for i in range(len(chordList) - sequenceLength):
        train_chords.append(chordList[i:i+sequenceLength])
        train_durations.append(durationList[i:i+sequenceLength])
        target_chords.append(chordList[i+1])
        target_durations.append(durationList[i+1])

In [13]:
train_chords = np.array(train_chords)
train_durations = np.array(train_durations)

In [14]:
n_samples = train_chords.shape[0]
n_chords = train_chords.shape[1]
n_durations = train_durations.shape[1]

In [15]:
input_dimensions = n_chords * sequenceLength

In [16]:
embed_dim = 64

In [17]:
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, LSTM
from tensorflow.keras import Model

In [18]:
chord_input = Input(shape = (None,))
duration_input = Input(shape = (None,))

In [19]:
chord_embedding = Embedding(n_chords, embed_dim, input_length = sequenceLength)(chord_input)
duration_embedding = Embedding(n_durations, embed_dim, input_length = sequenceLength)(duration_input)

In [20]:
merge_layer = Concatenate(axis=1)([chord_embedding, duration_embedding])

In [21]:
lstm_layer = LSTM(512, return_sequences=True)(merge_layer)
dense_layer = Dense(256)(lstm_layer)

In [22]:
chord_output = Dense(n_chords, activation='softmax')(dense_layer)
duration_output = Dense(n_durations, activation = 'softmax')(dense_layer)

In [23]:
lstm = Model(inputs = [chord_input, duration_input], outputs = [chord_output, duration_output])

In [24]:
lstm.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')

In [31]:
lstm.fit([train_chords, train_durations], [target_chords, target_durations], epochs=500, batch_size=64)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'})